In [ ]:
- get workspace
- get compute cluster
- Create Experiment
- Data sets
    - we can reuse already registred data sets
- Register Training script
- Register Measures 
- Logs
- Define Parameters
- Submit Experiment

- Register Model

- Deploy Model

- Test Model


Get workspace 

In [ ]:
import os
import azureml
from azureml.core import Workspace
print("Azure ML SDK Version: ", azureml.core.VERSION)
ws = Workspace.from_config()
print(ws.name)


Get Compute target: I already created a compute cluster
I only need to lookup for the existing cluster

In [ ]:
from azureml.core import ComputeTarget
cluster_name = "lm-dl-tc"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

print(compute_target)

Create New Experiment

In [ ]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='keras-mnist-cnn')
print(exp)

Create Data Sets

In [ ]:
from azureml.core.dataset import Dataset
web_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path=web_paths)

Register Data set : if not alraedy done

In [ ]:
dataset = Dataset.get_by_name(ws, 'mnist dataset')
if (dataset is None):
    print("Register new dataset")
    dataset = dataset.register(workspace=ws,
        name='mnist dataset',
        description='training and test dataset',
        create_new_version=True)
else:
    print("Dataset already exists")
# list the files referenced by mnist dataset
dataset.to_path() 

Define script:
Script folder
Script name

In [ ]:
#Script folder : where the training script will be located
script_folder = './scripts'
os.makedirs(script_folder, exist_ok=True)

#Script name
script_name= 'Mninst_CNN.py'

TensorFlow Estimator

In [ ]:
from azureml.train.dnn import TensorFlow
print(TensorFlow.DEFAULT_VERSION)


In [ ]:
# Parameters
script_params = {
    '--data-folder': dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 2,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.001
}

In [ ]:
# Create Estimator
est = TensorFlow(source_directory=script_folder,
                 entry_script=script_name,
                 script_params=script_params,
                 compute_target=compute_target,
                 pip_packages=['keras', 'matplotlib'],
                 use_gpu=True)

In [ ]:
run = exp.submit(est)

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()